# Visualisation des Donnnées du Covid-19 🦠 en France

## Chargement des libraries

In [1]:
import pandas as pd
import streamlit as st
import altair as alt

from altair import datum
from typing import Tuple

#lien de téléchargement du Dataset
URL_OPENCOVID19 = "https://raw.githubusercontent.com/opencovid19-fr/data/master/dist/chiffres-cles.csv"

## Chargement du Dataset et DataPrep

### Fonction Get Data

Fonction get_data(URL) permettant d'avoir à chaque exécution de l'algorithmes toutes les métriques calculées à jour

In [2]:
# make title
st.title("Visualisation des Donnnées du Covid-19 🦠 en France")


#@st.cache
def get_data(url: str = URL_OPENCOVID19) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    1 - Télécharger les données depuis la source GitHub
    2 - Transformer le tableau en dataframe
    3 - Renvoie df_covid19_fr (columns = ['date', 'type', 'nombre']) et
                df_covid19_region (columns = ['date', 'maille_nom', 'cas_confirmes', 'deces', 'delta_deces',
                                              'delta_cas_confirmes', 'fatality_rate', 'days_after_5_deaths',
                                               'days_after_50_confirmed']) 
    """
    # 1 - Téléchargement de la Data
    data = pd.read_csv(url)

    # 2 - Transformations pour obtenir df_covid19_fr (columns = ['date', 'type', 'nombre'])
    df_covid19_fr = data[
        (data.granularite == "pays") & (data.source_type == "sante-publique-france")
    ]
    # reshape df
    df_covid19_fr = pd.melt(
        df_covid19_fr,
        id_vars=["date"],
        value_vars=["cas_confirmes", "deces", "reanimation"],
        var_name="type",
        value_name="nombre",
    )
    df_covid19_fr = df_covid19_fr.fillna(value=0)

    # 2 - Transformation pour avoir les données par Régions df_covid19_region
    df_covid19_region = data[
        (data.granularite == "region")
        & (data.source_type == "agences-regionales-sante")
    ]
    df_covid19_region = df_covid19_region[
        ["date", "maille_nom", "cas_confirmes", "deces"]
    ]
    df_covid19_region = df_covid19_region.sort_values(by=["maille_nom", "date"])
    # calcul de delta_deces en faisant la diff entre les lignes de deces groupées par region
    df_covid19_region["delta_deces"] = df_covid19_region.groupby("maille_nom")[
        "deces"
    ].diff()
    # calcul de delta_cas_confirmes en faisant la diff entre les lignes de cas_confirmes groupées par region
    df_covid19_region["delta_cas_confirmes"] = df_covid19_region.groupby("maille_nom")[
        "cas_confirmes"
    ].diff()
    # Taux de mortalité/fatality_rate
    df_covid19_region["fatality_rate"] = (
        df_covid19_region["deces"] / df_covid19_region["cas_confirmes"]
    )
    # Nouvelle mésure nasée sur le nombre de jours après 5 deces/days after 5 deaths
    df_covid19_region["days_after_5_deaths"] = (
        df_covid19_region[df_covid19_region.deces > 5]
        .groupby("maille_nom")["deces"]
        .rank(method="first", ascending=True)
    )
    # Nouvelle mésure nasée sur le nombre de jours après 50 cas confirmés/days_after_50_confirmed
    df_covid19_region["days_after_50_confirmed"] = (
        df_covid19_region[df_covid19_region.cas_confirmes > 50]
        .groupby("maille_nom")["cas_confirmes"]
        .rank(method="first", ascending=True)
    )
    df_covid19_region.reset_index(drop=True)

    return df_covid19_region, df_covid19_fr

### Get Data

In [3]:
df_covid19_region, df_covid19_fr = get_data()

NumExpr defaulting to 8 threads.


In [4]:
df_covid19_region

,date,maille_nom,cas_confirmes,deces,delta_deces,delta_cas_confirmes,fatality_rate,days_after_5_deaths,days_after_50_confirmed
182,2020-02-27,Auvergne-Rhône-Alpes,2.0,0.0,NaN,NaN,0.000000,NaN,NaN
290,2020-03-02,Auvergne-Rhône-Alpes,32.0,0.0,0.0,30.0,0.000000,NaN,NaN
345,2020-03-03,Auvergne-Rhône-Alpes,36.0,0.0,0.0,4.0,0.000000,NaN,NaN
432,2020-03-04,Auvergne-Rhône-Alpes,51.0,0.0,0.0,15.0,0.000000,NaN,1.0
530,2020-03-05,Auvergne-Rhône-Alpes,63.0,0.0,0.0,12.0,0.000000,NaN,2.0
...,...,...,...,...,...,...,...,...,...
1086,2020-03-10,Île-de-France,283.0,4.0,3.0,0.0,0.014134,NaN,6.0
1230,2020-03-11,Île-de-France,512.0,4.0,0.0,229.0,0.007812,NaN,7.0
1366,2020-03-12,Île-de-France,605.0,4.0,0.0,93.0,0.006612,NaN,8.0
1486,2020-03-13,Île-de-France,876.0,NaN,NaN,271.0,NaN,NaN,9.0


In [5]:
df_covid19_fr

,date,type,nombre
0,2020-01-24,cas_confirmes,3.0
1,2020-01-25,cas_confirmes,3.0
2,2020-01-26,cas_confirmes,3.0
3,2020-01-27,cas_confirmes,3.0
4,2020-01-28,cas_confirmes,4.0
...,...,...,...
139,2020-03-24,reanimation,0.0
140,2020-03-25,reanimation,0.0
141,2020-03-26,reanimation,0.0
142,2020-03-27,reanimation,0.0


## create streamlit sidebar

In [6]:
regions = list(df_covid19_region.maille_nom.unique())
option = st.sidebar.selectbox("Visualisation: ", ("graph", "heatmap", "histo"))
check_box_table = st.sidebar.checkbox("Afficher les données")
check_box_analyse = st.sidebar.checkbox("Afficher l'analyse")

multiselection = st.sidebar.multiselect(
    "Selection des régions:", regions, default=regions
)
st.sidebar.info(
    "Merci à tous contributeurs du projet [opencovid19-fr](https://github.com/opencovid19-fr/data) pour leur travail de collecte des données officielles sur la progression de l'épidémie en France."
)


# get df_covid19_region based on region in multiselection

Retourne le DataFrame du covid_19 classé par région uniquement pour les éléménts (régions) sélectionnés dans le Sidebar

In [7]:
df_covid19_region = df_covid19_region[
    df_covid19_region["maille_nom"].isin(multiselection)
].sort_values(by=["maille_nom", "date"], ascending=[True, False])


Afficher les données ou pas - Si la case "Analyse les données" est active, afficher les données sinon ne rien afficher

In [8]:
if check_box_table:
    st.write(df_covid19_region)

## Graphe N°1: Evolution linéaire du nombre de déces et du nombre de cas confirmés

In [9]:
scale = alt.Scale(type="linear")
c_deces = (
        alt.Chart(df_covid19_region)
        .mark_line(point=True)
        .encode(
            alt.X("days_after_5_deaths"),
            alt.Y("deces", scale=scale),
            alt.Color("maille_nom"),
            tooltip=["days_after_5_deaths", "deces", "maille_nom"],
        )
        .interactive()
    )



c_deces


alt.Chart(...)

In [10]:
c_confirmed = (
    alt.Chart(df_covid19_region)
    .mark_line(point=True)
    .encode(
        alt.X("days_after_50_confirmed"),
        alt.Y("cas_confirmes", scale=scale),
        alt.Color("maille_nom"),
        tooltip=["days_after_5_deaths", "deces", "maille_nom"],
    )
    .interactive()
)
c_confirmed

alt.Chart(...)

In [11]:
if option == "graph":
    if st.checkbox("Log Scale"):
        scale = alt.Scale(type="log", domain=[10, 5000], clamp=True)
    else:
        scale = alt.Scale(type="linear")

    if check_box_analyse:
        st.info(
            "[03/22] Les régions Grand-Est, Ile-de-France et Haut-de-France sont les plus touchées par l'épidémie. "
            "Par ailleurs l'affiche en échelle Log, nous montre que l'ensemble des régions suivent la même croissance en terme d'évolution"
        )
    # make plot on nb of deces by regions
    c_deces = (
        alt.Chart(df_covid19_region)
        .mark_line(point=True)
        .encode(
            alt.X("days_after_5_deaths"),
            alt.Y("deces", scale=scale),
            alt.Color("maille_nom"),
            tooltip=["days_after_5_deaths", "deces", "maille_nom"],
        )
        .interactive()
    )

    # make plot on nb confirmed by regions
    c_confirmed = (
        alt.Chart(df_covid19_region)
        .mark_line(point=True)
        .encode(
            alt.X("days_after_50_confirmed"),
            alt.Y("cas_confirmes", scale=scale),
            alt.Color("maille_nom"),
            tooltip=["days_after_5_deaths", "deces", "maille_nom"],
        )
        .interactive()
    )

    st.altair_chart(c_deces, use_container_width=True)
    st.altair_chart(c_confirmed, use_container_width=True)

## Graphe N°2: Heatmap du nombre de déces et du nombre de cas confirmés

Nombres de nouveaux décès (delta_deces)

In [12]:
c_heatmap_deces = (
        alt.Chart(df_covid19_region)
        .mark_rect()
        .encode(
            alt.X("date"),
            alt.Y("maille_nom:N"),
            alt.Color("delta_deces:Q", scale=alt.Scale(scheme="lightmulti")),
            tooltip=["date", "maille_nom", "delta_deces"],
        )
        .interactive()
    )
c_heatmap_deces

alt.Chart(...)

Cas nouveaux cas confirmés (delta_cas_confirmes)

In [13]:
c_heatmap_confirmed = (
        alt.Chart(df_covid19_region)
        .mark_rect()
        .encode(
            alt.X("date"),
            alt.Y("maille_nom:N"),
            alt.Color("delta_cas_confirmes:Q", scale=alt.Scale(scheme="lightmulti")),
            tooltip=["date", "maille_nom", "delta_deces"],
        )
        .transform_filter((datum.delta_cas_confirmes >= 0))
        .interactive()
    )
c_heatmap_confirmed

alt.Chart(...)

Evolution

In [14]:
c_circle_evo = (
        alt.Chart(df_covid19_region)
        .mark_circle()
        .encode(
            alt.X("date"),
            alt.Y("maille_nom:N"),
            alt.Color("deces:Q", scale=alt.Scale(scheme="lightmulti")),
            alt.Size("cas_confirmes:N", bin=alt.Bin(maxbins=5)),
        )
    .interactive()
    )

c_circle_evo

alt.Chart(...)

elif option == "heatmap":
    if check_box_analyse:
        st.info(
            "[03/22] Les régions Grand-Est, Ile-de-France et Haut-de-France ont été les premières touchées par l'épidémie. "
        )

    # make heatmap on nb of deces by regions
    c_heatmap_deces = (
        alt.Chart(df_covid19_region)
        .mark_rect()
        .encode(
            alt.X("date"),
            alt.Y("maille_nom:N"),
            alt.Color("delta_deces:Q", scale=alt.Scale(scheme="lightmulti")),
            tooltip=["date", "maille_nom", "delta_deces"],
        )
        .interactive()
    )

    # make heatmap on nb of confirmed by regions
    c_heatmap_confirmed = (
        alt.Chart(df_covid19_region)
        .mark_rect()
        .encode(
            alt.X("date"),
            alt.Y("maille_nom:N"),
            alt.Color("delta_cas_confirmes:Q", scale=alt.Scale(scheme="lightmulti")),
            tooltip=["date", "maille_nom", "delta_deces"],
        )
        .transform_filter((datum.delta_cas_confirmes >= 0))
        .interactive()
    )
    c_circle_evo = (
        alt.Chart(df_covid19_region)
        .mark_circle()
        .encode(
            alt.X("date"),
            alt.Y("maille_nom:N"),
            alt.Color("deces:Q", scale=alt.Scale(scheme="lightmulti")),
            alt.Size("cas_confirmes:N", bin=alt.Bin(maxbins=5)),
        )
    )

    st.altair_chart(c_heatmap_deces, use_container_width=True)
    st.altair_chart(c_heatmap_confirmed, use_container_width=True)
    st.altair_chart(c_circle_evo, use_container_width=True)


## Graphe N°3: Histogramme évolution en France et taux de mortalité

Evolution en France

In [16]:
c_evo_fr = (
        alt.Chart(df_covid19_fr)
        .mark_bar()
        .encode(x="date", y=alt.Y("nombre"), color="type")
        .transform_filter((datum.date >= "2020-03-01"))
        .interactive()
    )
c_evo_fr

alt.Chart(...)

Taux de mortalié en France

In [19]:
c_histo_fatality_rate = (
        alt.Chart(df_covid19_region)
        .mark_bar()
        .encode(alt.Y("maille_nom:N", sort="-x"), alt.X("mean(fatality_rate):Q"),)
        .transform_filter((datum.date >= "2020-03-10"))
        .interactive()
    )
c_histo_fatality_rate

alt.Chart(...)

elif option == "histo":

    # make histo on evo in france
    c_evo_fr = (
        alt.Chart(df_covid19_fr)
        .mark_bar()
        .encode(x="date", y=alt.Y("nombre"), color="type")
        .transform_filter((datum.date >= "2020-03-01"))
    )
    # make histo on fatality rate in france
    c_histo_fatality_rate = (
        alt.Chart(df_covid19_region)
        .mark_bar()
        .encode(alt.Y("maille_nom:N", sort="-x"), alt.X("mean(fatality_rate):Q"),)
        .transform_filter((datum.date >= "2020-03-10"))
    )

    if check_box_analyse:
        st.info(
            "[03/22] Evolution par jour du nombre de cas, de réanimation et de décès. Chaque jour, les valeurs augmentent."
        )

    st.altair_chart(c_evo_fr, use_container_width=True)

    if check_box_analyse:
        st.info(
            "[03/22] Le graphique ci-dessous estime le taux de mortalité de l'épidémie Covid-19 dans les différentes régions de France. "
            "⚠️ A  noter que cette valeur dépend du nombre de test et par conséquent surestime la véritable valeur du taux de mortalité de l'épidémie"
        )
    st.altair_chart(c_histo_fatality_rate, use_container_width=True)

